In [3]:
import pandas as pd
import numpy as np
test_data = pd.read_csv("/Users/hyunwoo/Downloads/nsmc-master/ratings_test.txt",sep ="\t")
train_data = pd.read_csv("/Users/hyunwoo/Downloads/nsmc-master/ratings_train.txt",sep="\t")

In [12]:
train_data.isnull().sum()

id          0
document    5
label       0
dtype: int64

In [13]:
train_data.dropna(inplace=True)

In [16]:
train_data.reset_index(inplace=True,drop=True)
train_target = train_data['label']

In [19]:
train_data['label'].value_counts() # not class imbalanced 

0    75170
1    74825
Name: label, dtype: int64

In [26]:
import matplotlib.pyplot as plt
%matplotlib inline


Object `pyplot` not found.


In [35]:
import re
re.sub('[^ㄱ-힣]', '', train_data['document'].values[4])

'사이몬페그의익살스런연기가돋보였던영화스파이더맨에서늙어보이기만했던커스틴던스트가너무나도이뻐보였다'